# Data Analysis of the Disney Datasets
By: Audrey Warner

# Introduction

## Question of Interest
For this analysis, I am interested in investigating the gross earnings and directors of Disney movies. I am interested to see which director has made the most profit for Disney over the years, and what genre of film is the most lucrative. This may give insight into which genres are most popular.  

## Dataset Description

The Walt Disney Company is a multi-billion dollar entertainment and media company. The Walt Disney Company has produced a wide variety of family-friendly films since its inception in 1923.  

The Disney dataset is made up of 5 tables, `disney_movies_total_gross.csv`, `disney_revenue_1991-2016.csv`, `disney-characters.csv`, `disney-director.csv`, `disney-voice-actors.csv`. The tables are stored in `.csv` files and contain information about Disney movies, specifically revenue, characters, directors and actors. I have chosen to complete an analysis of the total gross earnings and director tables:

* **disney_movies_total_gross.csv**

    * This file contains information on the gross revenue of Disney films, including the movie's release date, genre, its MPAA rating, its total gross earnings, and its earnings adjusted for inflation.

* **disney-director.csv**

    * This file shows the director of each Disney film.

# Methods and Results

In [1]:
# Import libraries for analysis
import altair as alt
import pandas as pd

# Import the Disney data files, convert release_date column to dtype datetime64

movies_total_gross = pd.read_csv(
    "data/disney_movies_total_gross.csv", parse_dates=["release_date"]
)
directors = pd.read_csv("data/disney-director.csv")

View the first 5 rows of each dataframe:

In [2]:
# View the first 5 rows of dataframe
movies_total_gross.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,1940-02-09,Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,1940-11-13,Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,1946-11-12,Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,1950-02-15,Drama,G,"$85,000,000","$920,608,730"


Table 1. First 5 rows of the movies_total_gross dataframe

In [3]:
# View the first 5 rows of dataframe
directors.head()

,name,director
0,Snow White and the Seven Dwarfs,David Hand
1,Pinocchio,Ben Sharpsteen
2,Fantasia,full credits
3,Dumbo,Ben Sharpsteen
4,Bambi,David Hand


Table 2. First 5 rows of the directors dataframe

In [4]:
# View the dtype of each column
movies_total_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   movie_title               579 non-null    object        
 1   release_date              579 non-null    datetime64[ns]
 2   genre                     562 non-null    object        
 3   MPAA_rating               523 non-null    object        
 4   total_gross               579 non-null    object        
 5   inflation_adjusted_gross  579 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 27.3+ KB


Table 3. Dtypes of each column in the movies_total_gross dataframe

In [5]:
# View the dtype of each column
directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      56 non-null     object
 1   director  56 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


Table 4. Dtypes of each column in the directors dataframe

The movies_total_gross dataset needs to be tidied before analysis. To start, I will remove symbols $ and , so that revenue columns can be recognized as having numerical values, then they can be converted to integer type columns. 

In [6]:
movies_total_gross = movies_total_gross.assign(
    total_gross=movies_total_gross["total_gross"].astype("str")
)
movies_total_gross = movies_total_gross.assign(
    total_gross=movies_total_gross["total_gross"].str.strip("$")
)
movies_total_gross = movies_total_gross.assign(
    total_gross=movies_total_gross["total_gross"].str.replace(",", "")
)
movies_total_gross = movies_total_gross.assign(
    inflation_adjusted_gross=movies_total_gross["inflation_adjusted_gross"].astype(
        "str"
    )
)
movies_total_gross = movies_total_gross.assign(
    inflation_adjusted_gross=movies_total_gross["inflation_adjusted_gross"].str.strip(
        "$"
    )
)
movies_total_gross = movies_total_gross.assign(
    inflation_adjusted_gross=movies_total_gross["inflation_adjusted_gross"].str.replace(
        ",", ""
    )
)
movies_total_gross = movies_total_gross.sort_values("release_date")
movies_total_gross = movies_total_gross.assign(
    total_gross=movies_total_gross["total_gross"].astype("int")
)
movies_total_gross = movies_total_gross.assign(
    inflation_adjusted_gross=movies_total_gross["inflation_adjusted_gross"].astype(
        "int"
    )
)
movies_total_gross

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485,5228953251
1,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808
3,Song of the South,1946-11-12,Adventure,G,65000000,1078510579
4,Cinderella,1950-02-15,Drama,G,85000000,920608730
...,...,...,...,...,...,...
574,The Light Between Oceans,2016-09-02,Drama,PG-13,12545979,12545979
575,Queen of Katwe,2016-09-23,Drama,PG,8874389,8874389
576,Doctor Strange,2016-11-04,Adventure,PG-13,232532923,232532923
577,Moana,2016-11-23,Adventure,PG,246082029,246082029


Table 5. Cleaned movies_total_gross dataframe 

In [7]:
# Use groupby to view the inflation-adjusted gross revenue for each genre

genre_group = pd.DataFrame(
    movies_total_gross.groupby("genre")["inflation_adjusted_gross"]
    .sum()
    .sort_values(ascending=False)
)
genre_group = genre_group.reset_index()
genre_group

,genre,inflation_adjusted_gross
0,Adventure,24561266158
1,Comedy,15409526913
2,Musical,9657565776
3,Drama,8195804484
4,Action,5498936786
5,Thriller/Suspense,2151690954
6,Romantic Comedy,1788872933
7,Western,516709946
8,Documentary,203488418
9,Black Comedy,156730475


Table 6. Inflation-adjusted gross revenue grouped by genre

In [8]:
# Use altair to create a bar plot

genre_plot = (
    alt.Chart(genre_group, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("genre:O", title="Genre", sort="-y"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Gross Revenue (USD)"),
    )
    .properties(
        title="Inflation-Adjusted Gross Revenue of Disney Films from 1937 to 2016"
    )
)
genre_plot

alt.Chart(...)

Figure 1. Plot of inflation-adjusted gross revenue of Disney films by genre between 1937 and 2016.

In [9]:
# Join datasets using merge

disney_director_revenue = movies_total_gross.merge(
    directors, left_on="movie_title", right_on="name", how="inner", indicator=False
)
disney_director_revenue = disney_director_revenue.drop(columns="name")
disney_director_revenue

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,director
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485,5228953251,David Hand
1,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052,Ben Sharpsteen
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808,full credits
3,Cinderella,1950-02-15,Drama,G,85000000,920608730,Wilfred Jackson
4,Cinderella,2015-03-13,Drama,PG,201151353,201151353,Wilfred Jackson
5,Lady and the Tramp,1955-06-22,Drama,G,93600000,1236035515,Hamilton Luske
6,Sleeping Beauty,1959-01-29,Drama,NaN,9464608,21505832,Clyde Geronimi
7,101 Dalmatians,1961-01-25,Comedy,G,153000000,1362870985,Wolfgang Reitherman
8,101 Dalmatians,1996-11-27,Comedy,G,136189294,258728898,Wolfgang Reitherman
9,The Sword in the Stone,1963-12-25,Adventure,NaN,22182353,153870834,Wolfgang Reitherman


Table 7. Merged dataframe showing information about each Disney film's total gross revenue and its director

Because the Disney film catalogue is so large, I made a function that will allow us to look up specific information about Disney films. For example, using this function, we can look up who directed the film 'Moana'.   

In [10]:
from df_finder import df_finder

Let's find out what this function does. 

In [11]:
?df_finder

Signature: df_finder(dataframe, dataframe_column, column_value, show)
Docstring:
Given a dataframe, return a smaller sample of the dataframe
sampling N rows from each specified group

Parameters
----------
dataframe : pandas.core.frame.DataFrame
    The dataframe to sample from
dataframe_column : str
    The column of interest
column_value : any
    Identifying information of a particular row
show :
    Other column information of that particular row

Returns
-------
pandas.core.frame.DataFrame
    The new sampled dataframe

Examples
--------
File:      ~/prog-python-data-science-students/release/final_project/df_finder.py
Type:      function


In [12]:
df_finder(disney_director_revenue,'movie_title','Moana',['movie_title','director'])

,movie_title,director
48,Moana,Ron Clements


Table 8. Output of the df_finder function, showing the director of the film 'Moana'

In [13]:
# View only rows that are adventure films
director_adventure = (
    disney_director_revenue.loc[disney_director_revenue["genre"] == "Adventure"]
    .reset_index()
    .drop(columns="index")
)
director_adventure

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,director
0,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052,Ben Sharpsteen
1,The Sword in the Stone,1963-12-25,Adventure,NaN,22182353,153870834,Wolfgang Reitherman
2,The Jungle Book,1994-12-25,Adventure,PG,44342956,88930321,Wolfgang Reitherman
3,The Jungle Book,2016-04-15,Adventure,PG,364001123,364001123,Wolfgang Reitherman
4,The Rescuers,1977-06-22,Adventure,NaN,48775599,159743914,Wolfgang Reitherman
5,The Black Cauldron,1985-07-24,Adventure,NaN,21288692,50553142,Ted Berman
6,The Great Mouse Detective,1986-07-02,Adventure,NaN,23605534,53637367,Ron Clements
7,Oliver & Company,1988-11-18,Adventure,G,49576671,102254492,George Scribner
8,The Little Mermaid,1989-11-15,Adventure,G,111543479,223726012,Ron Clements
9,The Rescuers Down Under,1990-11-16,Adventure,G,27931461,55796728,Mike Gabriel


Table 9. A dataframe showing only Adventure films

In [14]:
grouped_director_adventure = director_adventure.groupby(by='director')
grouped_director_adventure.size()

director
Barry Cook             1
Ben Sharpsteen         1
Byron Howard           1
Chris Buck             2
Chris Sanders          1
Clyde Geronimi         1
Don Hall               1
Gary Trousdale         2
George Scribner        1
Mark Dindal            2
Mike Gabriel           2
Nathan Greno           1
Ralph Zondag           1
Rich Moore             1
Robert Walker          1
Roger Allers           1
Ron Clements           6
Stephen J. Anderson    2
Ted Berman             1
Wolfgang Reitherman    4
dtype: int64

Table 10. The number of Adventure films directed by each director

From this analysis, we see that Ron Clements has directed the most adventure films, at 6. Shown below are the adventure films he has directed.

In [15]:
ron_clements = grouped_director_adventure.get_group("Ron Clements")
ron_clements

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,director
6,The Great Mouse Detective,1986-07-02,Adventure,NaN,23605534,53637367,Ron Clements
8,The Little Mermaid,1989-11-15,Adventure,G,111543479,223726012,Ron Clements
13,Hercules,1997-06-13,Adventure,G,99112101,182029412,Ron Clements
20,Treasure Planet,2002-11-27,Adventure,PG,38120554,55189145,Ron Clements
24,The Princess and the Frog,2009-11-25,Adventure,G,104400899,116316457,Ron Clements
32,Moana,2016-11-23,Adventure,PG,246082029,246082029,Ron Clements


Table 11. A dataframe showing only Adventure films directed by Ron Clements

In [16]:
# Sort dataframe in descending order
ron_clements_sorted = (
    ron_clements.sort_values(by="inflation_adjusted_gross", ascending=False)
    .reset_index()
    .drop(columns="index")
)
ron_clements_sorted

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,director
0,Moana,2016-11-23,Adventure,PG,246082029,246082029,Ron Clements
1,The Little Mermaid,1989-11-15,Adventure,G,111543479,223726012,Ron Clements
2,Hercules,1997-06-13,Adventure,G,99112101,182029412,Ron Clements
3,The Princess and the Frog,2009-11-25,Adventure,G,104400899,116316457,Ron Clements
4,Treasure Planet,2002-11-27,Adventure,PG,38120554,55189145,Ron Clements
5,The Great Mouse Detective,1986-07-02,Adventure,NaN,23605534,53637367,Ron Clements


Table 12. A dataframe showing only Adventure films directed by Ron Clements, sorted in descending order by inflation-adjusted gross revenue

In [17]:
# Use altair to create a bar plot
ron_clements_plot = (
    alt.Chart(ron_clements_sorted, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("movie_title:O", title="Movie Title", sort="-y"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Gross Revenue USD"),
    )
    .properties(
        title="Revenue Generated from Adventure Films Directed by Ron Clements"
    )
)
ron_clements_plot

alt.Chart(...)

Figure 2. Plot of inflation-adjusted revenue generated from films directed by Ron Clements. 

From the plot above, we see that Ron Clements has directed 6 Adventure films. The film that generated the most revenue was Moana, and the least was The Great Mouse Detective. It is interesting that Moana is also his most recent film. There may be other factors influencing revenue, such as the amount of promotion or budget. 

Finally, I will determine the total inflation-adjusted revenue that Ron Clements has generated for Disney from the six Adventure films he directed.  

In [18]:
ron_clements_revenue = ron_clements_sorted['inflation_adjusted_gross'].sum()
ron_clements_revenue

876980422

# Discussion
For my project, I worked with the Disney dataset to see which directors and genres generated the most revenue. Initial analysis revealed that Adventure films generated the most revenue, which doesn't surprise me. 

When I looked at the directors of Disney's Adventure films, I was surprised that most directed only one or two films, except for Ron Clements, who directed 6. I wanteed to see which of his films generated the most revenue, and the total revenue he generated for Disney. 

Because revenue can indicate a film's popularity, this analysis gives us insight into the most popular genre and which director has been the most successful in directing Adventure films to date.

One question to look into in more depth would be how many films of each genre each director has directed, and how it has changed since 1937. 

# References
Not all the work in this workbook is original. I did adapt certain functions and tests that were relevant to my analysis of the Disney datasets from course content and past assignments that I submitted.

## Resources Used
* The data used in this analysis were obtained from [data.world](https://data.world/kgarrett/disney-character-success-00-16)  
* I came up with the data visualization and research question independently
 